In [43]:
import pandas as pd
import polars as pl
from pathlib import Path
import duckdb
from src.enbridgescrape import formatOA, paths

In [2]:
metaPath = paths.downloads / 'MetaData'
configsPath = paths.configs 

In [3]:
dfList= []
for filePath in metaPath.iterdir():
    tempDf = pd.read_csv(filePath)
    dfList.append(tempDf)

In [4]:
df_TSP=pl.LazyFrame(pd.concat(dfList)[['TSP','TSP Name']].drop_duplicates())\
    .with_columns(
        pl.col('TSP Name').alias('TSP_Name')
    )

In [5]:
metaDf = pl.scan_csv( metaPath / "*.csv")

In [ ]:
with duckdb.connect(configsPath / "GFPipes.db") as con:
    con.execute("""
                CREATE SEQUENCE IF NOT EXISTS GFPipeID_sequence
                INCREMENT BY 1
                MINVALUE 100
                MAXVALUE 999
                START WITH 100
                """)
    con.execute("""
        CREATE TABLE IF NOT EXISTS GFPipeID_table (
            id INTEGER PRIMARY KEY DEFAULT nextval('GFPipeID_sequence'),
            TSP INTEGER UNIQUE NOT NULL,
            TSP_Name VARCHAR UNIQUE NOT NULL
        );
    """)
    
    con.execute("""
        MERGE INTO GFPipeID_table AS target
        USING df_TSP AS source
        ON target.TSP = source.TSP and target.TSP_Name = source.TSP_Name
        WHEN MATCHED THEN
            UPDATE SET TSP = source.TSP, TSP_Name = source.TSP_Name
        WHEN NOT MATCHED THEN
            INSERT (TSP,TSP_Name) VALUES (source.TSP, source.TSP_Name);
    """)    

# with duckdb.connect(configsPath / "GFPipes.db") as con:
    # con.execute("INSERT INTO GFPipeID_table (TSP,TSP_Name ) SELECT * FROM df_TSP")

In [2]:
import duckdb

with duckdb.connect("/Users/saijagadeeshyadavalli/Coding/Particles/GasFundies/enbridgeGitDump/src/Models/GFPipes.db") as con:
    res=con.execute("""SELECT * from GFPipes_table""").fetchall()
res

[(100, 835460478, 'Egan Hub Storage, LLC', 'Enbridge'),
 (101, 966981599, 'Garden Banks Gas Pipeline, LLC', 'Enbridge'),
 (102, 80151107, 'NEXUS Gas Transmission, LLC (U.S.)', 'Enbridge'),
 (103, 7921323, 'East Tennessee Natural Gas, LLC', 'Enbridge'),
 (104, 827593182, 'Steckman Ridge, LP', 'Enbridge'),
 (105, 81787256, 'Maritimes & Northeast Pipeline, L.L.C. (U.S.)', 'Enbridge'),
 (106, 791204600, 'Tres Palacios Gas Storage LLC', 'Enbridge'),
 (107, 791287274, 'Saltville Gas Storage Company L.L.C.', 'Enbridge'),
 (108, 968195714, 'Big Sandy Pipeline, LLC', 'Enbridge'),
 (109, 614834559, 'Bobcat Gas Storage', 'Enbridge'),
 (110, 808264746, 'Southeast Supply Header, LLC', 'Enbridge'),
 (111, 176824738, 'Nautilus Pipeline Company, L.L.C.', 'Enbridge'),
 (112, 7932908, 'Texas Eastern Transmission, LP', 'Enbridge'),
 (113, 962767138, 'Mississippi Canyon Gas Pipeline, LLC', 'Enbridge'),
 (114, 6951446, 'Algonquin Gas Transmission, LLC', 'Enbridge'),
 (115, 930373670, 'Sabal Trail Transmiss

In [23]:
with duckdb.connect(configsPath / "GFPipes.db") as con:
    res =   pl.LazyFrame(data=con.execute("""
                                        SELECT * from GFPipeID_table
                                        """,
                                    ).fetchall(), 
                        schema=['id','TSP','TSP_Name'], orient='row')\
                        .with_columns(
                            pl.col('id').cast(pl.String).alias('GFPipeID'))\
                        .select(['GFPipeID','TSP'])

In [44]:
dfOA = formatOA()

ColumnNotFoundError: unable to find column "GFLOC"; valid columns: ["CycleDesc", "EffGasDayTime", "Loc", "LocName", "LocZn", "FlowInd", "LocPurpDesc", "IT", "AllQtyAvail", "DesignCapacity", "OperatingCapacity", "TotalScheduledQuantity", "OperationallyAvailableCapacity", "TSP", "LocSegment", "QtyReason", "Timestamp"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
SELECT [col("Cycle_Desc").alias("CycleDesc"), col("Eff_Gas_Day").alias("EffGasDayTime"), col("Loc"), col("Loc_Name").alias("LocName"), col("Loc_Zn").alias("LocZn"), col("Flow_Ind_Desc").alias("FlowInd"), col("Loc_Purp_Desc").alias("LocPurpDesc"), col("IT"), col("All_Qty_Avail").alias("AllQtyAvail"), col("Total_Design_Capacity").alias("DesignCapacity"), col("Operating_Capacity").alias("OperatingCapacity"), col("Total_Scheduled_Quantity").alias("TotalScheduledQuantity"), col("Operationally_Available_Capacity").alias("OperationallyAvailableCapacity"), col("TSP"), col("LocSegment"), col("QtyReason"), col("Ti

In [10]:
from datetime import datetime

import functools
import operator

OA_path = paths.downloads / 'OA'
selectedCols = ['Cycle_Desc', 'Eff_Gas_Day', 'Loc', 'Loc_Name', 'Loc_Zn', 'Flow_Ind_Desc', 'Loc_Purp_Desc', 'IT', 'All_Qty_Avail',
                'Total_Design_Capacity', 'Operating_Capacity', 'Total_Scheduled_Quantity', 'Operationally_Available_Capacity']


In [11]:
def batchDateParse(inSeries: pl.Series) -> pl.Series:
    return pl.Series(map(lambda inString: datetime.strptime(inString, "%m-%d-%Y").date(), inSeries))


def batchFloatParse(inSeries: pl.Series) -> pl.Series:
    return pl.Series(map(lambda inString: float(inString.replace(',', '')), inSeries))


def batchFIMapper(inSeries: pl.Series) -> pl.Series:
    flow_Map = {
        'Delivery': 'D',
        'Receipt': 'R',
        'Storage Injection': 'D',
        'Storage Withdrawal': 'R'
    }
    return pl.Series(map(lambda inString: flow_Map.get(inString, 'D'), inSeries))



In [33]:
df = pl.scan_csv(OA_path / "*_OA_*.csv",
                    glob=True,
                    has_header=True,
                    )\
    .select([*selectedCols, 'TSP'])\
    .with_columns(pl.col("TSP").cast(pl.Int64))\
    .filter(
    ~functools.reduce(
        operator.or_,
        map(lambda col: pl.col(col).is_null(), selectedCols)
    ))\
    .join(res, on='TSP', how='left')

In [41]:
df.with_columns(
        pl.col("Eff_Gas_Day").map_batches(
            batchDateParse, return_dtype=pl.Date),
        pl.col("Total_Design_Capacity").map_batches(
            batchFloatParse, return_dtype=pl.Float64),
        pl.col("Operating_Capacity").map_batches(
            batchFloatParse, return_dtype=pl.Float64),
        pl.col("Total_Scheduled_Quantity").map_batches(
            batchFloatParse, return_dtype=pl.Float64),
        pl.col("Operationally_Available_Capacity").map_batches(
            batchFloatParse, return_dtype=pl.Float64),
        pl.col("Flow_Ind_Desc").map_batches(
            batchFIMapper, return_dtype=pl.String),
        pl.lit(None).cast(pl.String).alias('LocSegment'),
        pl.lit(None).cast(pl.String).alias('QtyReason'),
        pl.lit(datetime.now()).cast(
            pl.Datetime).alias('Timestamp')
    )\
        .rename({
            'Cycle_Desc': 'CycleDesc',
            'Eff_Gas_Day': 'EffGasDayTime',
            'Loc_Name': 'LocName',
            'Loc_Zn': 'LocZn',
            'Flow_Ind_Desc': 'FlowInd',
            'Loc_Purp_Desc': 'LocPurpDesc',
            'All_Qty_Avail': 'AllQtyAvail',
            'Total_Design_Capacity': 'DesignCapacity',
            'Operating_Capacity': 'OperatingCapacity',
            'Total_Scheduled_Quantity': 'TotalScheduledQuantity',
            'Operationally_Available_Capacity': 'OperationallyAvailableCapacity',
            # 'TSP_Name': 'PipelineName'
        })\
        .select([
            'EffGasDayTime',
            # 'PipelineName',
            'CycleDesc',
            'LocPurpDesc',
            'Loc',
            'LocName',
            'LocZn',
            'LocSegment',
            'DesignCapacity',
            'OperatingCapacity',
            'TotalScheduledQuantity',
            'OperationallyAvailableCapacity',
            'IT',
            'FlowInd',
            'AllQtyAvail',
            'QtyReason',
            'Timestamp',
            'GFPipeID',
            'TSP'
        ]).collect()

EffGasDayTime,CycleDesc,LocPurpDesc,Loc,LocName,LocZn,LocSegment,DesignCapacity,OperatingCapacity,TotalScheduledQuantity,OperationallyAvailableCapacity,IT,FlowInd,AllQtyAvail,QtyReason,Timestamp,GFPipeID,TSP
date,str,str,str,str,str,str,f64,f64,f64,f64,str,str,str,str,datetime[μs],str,i64
2022-11-14,"""INTRDY_2022-11-15_0900""","""Delivery Location""","""70004""","""Dominion Energy Transmission, …","""2""",null,593830.0,600490.0,9556.0,590934.0,"""N""","""D""","""Y""",null,2025-11-13 21:13:56.225080,"""112""",7932908
2022-11-14,"""INTRDY_2022-11-15_0900""","""Delivery Location""","""70011""","""Columbia Gas of PA. - Eagle,…","""3""",null,291789.0,444405.0,125122.0,319283.0,"""N""","""D""","""Y""",null,2025-11-13 21:13:56.225080,"""112""",7932908
2022-11-14,"""INTRDY_2022-11-15_0900""","""Delivery Location""","""70017""","""Indiana Gas - Greensburg, IN""","""2""",null,21573.0,21573.0,2008.0,19565.0,"""N""","""D""","""Y""",null,2025-11-13 21:13:56.225080,"""112""",7932908
2022-11-14,"""INTRDY_2022-11-15_0900""","""Delivery Location""","""70018""","""Indiana Gas - Seymour, IN""","""2""",null,57331.0,58910.0,400.0,58510.0,"""N""","""D""","""Y""",null,2025-11-13 21:13:56.225080,"""112""",7932908
2022-11-14,"""INTRDY_2022-11-15_0900""","""Delivery Location""","""70020""","""Equitrans - Waynesburg, PA""","""2""",null,5011.0,5488.0,0.0,5488.0,"""N""","""D""","""Y""",null,2025-11-13 21:13:56.225080,"""112""",7932908
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-11-13,"""INTRDY_2025-11-13_1801""","""Delivery Location""","""79980""","""UNIONTOWN NORTH""","""2""",null,677000.0,677000.0,20420.0,656580.0,"""N""","""D""","""Y""",null,2025-11-13 21:13:56.225080,"""112""",7932908
2025-11-13,"""INTRDY_2025-11-13_1801""","""Receipt Location""","""79980""","""UNIONTOWN NORTH""","""2""",null,552000.0,552000.0,20420.0,531580.0,"""N""","""R""","""Y""",null,2025-11-13 21:13:56.225080,"""112""",7932908
2025-11-13,"""INTRDY_2025-11-13_1801""","""Delivery Location""","""79993""","""NEXUS INTERCONNECT WITH TETLP,…","""2""",null,637559.0,637559.0,0.0,637559.0,"""N""","""D""","""Y""",null,2025-11-13 21:13:56.225080,"""112""",7932908


In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('src/Models/EnbridgeMetaData.csv')

In [6]:
df

,GFPipeID,TSP,Loc,Loc Name,Loc St Abbrev,Loc Cnty,Loc Zone,Dir Flo,Loc Stat Ind,Loc Type Ind,...,Up/Dn Ind,Up/Dn Name,Up/Dn ID,Up/Dn ID Prop,Up/Dn FERC CID Ind,Up/Dn FERC CID,Up/Dn Loc,Up/Dn Loc Name,Loc Fuel Area,ParentPipe
0,100,835460478,40111,TARGA LOUISIANA FIELD SERVICES LLC,LA,ACADIA,NaN,D,A,INT,...,Y,TARGA MIDSTREAM SERVICES LLC,\t801475463,NaN,N,NaN,NaN,NaN,NaN,Enbridge
1,100,835460478,40117,"NRG-BAYOU COVE PEAKING POWER, LLC-ACADIA PSH., LA",LA,ACADIA,NaN,D,A,END,...,Y,PELICAN POWER LLC,\t119183265,NaN,N,NaN,NaN,NaN,NaN,Enbridge
2,100,835460478,45101,"ANR - STORAGE, LA (D40102/R40101)",LA,ACADIA,NaN,B,A,INT,...,Y,ANR PIPELINE COMPANY,\t006958581,NaN,Y,C000623,\t186900/186899,EGAN (RECEIPT) INT/EGAN (DELIVERY) INT,NaN,Enbridge
3,100,835460478,45103,"COLUMBIA - STORAGE, LA (D40104/R40103)",LA,ACADIA,NaN,B,A,INT,...,Y,"COLUMBIA GULF TRANSMISSION, LLC",\t007854581,NaN,Y,C000307,\t4123,EGAN STOR-ACADIA-REC,NaN,Enbridge
4,100,835460478,45105,"TENN. GAS - STORAGE, LA (D40106/40105)",LA,ACADIA,NaN,B,A,INT,...,Y,"TENNESSEE GAS PIPELINE COMPANY, L.L.C.",\t001939164,NaN,Y,C000020,\t420822,EGAN HUB/TGP EGAN GAS STORAGE ACADI,NaN,Enbridge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,115,930373670,96108,TECO WILDWOOD,FL,SUMTER,NaN,D,A,INT,...,Y,PEOPLES GAS SYSTEM,\t,29783.0,N,NaN,NaN,NaN,NaN,Enbridge
1779,115,930373670,98000,STT PAL POINT,AL,TALLAPOOSA,NaN,R,A,VIR,...,N,NaN,\t,NaN,N,NaN,NaN,NaN,NaN,Enbridge
1780,115,930373670,98105,T-85 POOL,AL,CHOCTAW,NaN,R,A,STA,...,Y,"TRANSCONTINENTAL GAS PIPE LINE COMPANY, LLC",\t007933021,NaN,Y,C000654,\t9009546,SABAL TRAIL TRANSFER POINT,NaN,Enbridge
1781,115,930373670,98962,GULF SOUTH SCOTT MOUNTAIN,AL,CHOCTAW,NaN,R,A,STA,...,Y,"GULF SOUTH PIPELINE COMPANY, LLC",\t078444247,NaN,Y,C000591,\t023169,ROCK SPRINGS (TO TRANSCO/SABAL TRAIL),NaN,Enbridge


In [7]:
import duckdb

In [8]:
with duckdb.connect('src/Models/EnbridgeMeta.db') as con:
    con.execute("""
        CREATE TABLE IF NOT EXISTS EnbridgeSegmentsTable AS SELECT * FROM df;
    """)